Solve given problem using genetic algorithm.

### Data Inputs:

- excess: 15-minute time-series data on producer energy supply. Columns are producer IDs.
- deficit: 15-minute time-series data on consumer energy demand. Columns are consumer IDs.

Each consumer can receive energy from up to 5 producers.

***Preferences are integers***:

- 5 = highest priority,
- 1 = lowest priority,
- 0 = no connection.

### Model Requirements:

1. **Decision Variables**:

- Priority matrix, Subscribers have preference vectors for up to 5 producers.
- Weight matrix, according which producers distribute energy

2. **Constraints**:

- A consumer can only receive energy from producers they are connected to (preference > 0).
- A consumer can only receive energy up to their demand.
- A producer can only distribute up to their available supply.
- Respect the fixed limit of 5 connections per consumer.
- Honor preference ordering if possible (e.g. prioritize higher-ranked producers).

3. **Objective Function**:

- Minimize the total unredistributed energy across all timestamps:
- Energy left with producers due to no eligible or matching demand.
- Unmet demand from consumers.

4. **Iteration**:

- The model must simulate distribution for each 15-minute interval (2880 time steps).
- After each time step:
- Update producer availability and consumer demand.
- Re-apply constraints and solve again (unless a single larger model across time steps is more efficient).

### Output:

- A feasible weight matrix and priority matrix
- Total unredistributed energy over the full month.
- A breakdown of which consumers didn’t receive their full demand and which producers had unused energy. All of this output to a file for later analysis.

### Details:

Producers have weight vectors for distributing energy. Subscribers have preference vectors for up to 5 producers.The function evaluates the energy distribution for each timestamp, calculates unredistributed energy, and tracks remaining demand for each subscriber.

### Process Simulation:

Every 15 minutes (96 times/day, ~2880 times/month), the system redistributes energy:

- Producers distribute energy according to their weight vectors.
- Subscribers take energy based on their preferences if they have unmet demand.
- Track unredistributed energy (demand that couldn't be met).


I have to make a simulation like this:

```
For each of 5 iterations:
    For each consumer:
        1. gather producers
        2. calculate vector of possible allocation
        3. For each producer based on priority:
           1. give energy to consumer if needed
           2. Adjust your available energy
-> collect unused energy as a fitness.
```

In [ ]:
import numpy as np

def control_simulation(excess_t, deficit_t, weights, preferences, rounds=5):
    supply = excess_t.copy()
    demand = deficit_t.copy()

    allocations = np.zeros((rounds, len(supply), len(demand)))


    for r in range(rounds):
        print(f"\n=== Round {r + 1} ===")

        # Each producer distributes supply according to static weights
        producer_offer = np.zeros(( len(supply), len(demand)))
        for i, producer in enumerate(supply):
            if supply[producer] > 0:
                producer_offer[i] = supply[producer] * weights[i]

        # Consumers take energy from preferred producers
        for j, consumer in enumerate(demand):
            if demand[consumer] <= 0:
                continue  # Already fulfilled

            for pref in preferences[j]:
                if pref == -1:
                    continue

                offer = producer_offer[pref][j]
                if offer <= 0:
                    continue

                take = min(offer, demand[consumer])

                # Allocate and update
                allocations[r][pref][j] += take
                producer_offer[pref][j] -= take
                supply[pref] -= take
                demand[j] -= take

                print(f"Consumer {j} takes {take:.1f} from Producer {pref}")

                if demand[j] <= 0:
                    break  # Move to next consumer

In [ ]:
import numpy as np

# Setup
num_producers = 2
num_consumers = 8
rounds = 5

# Supply per producer
producer_supply = np.array([15., 16.])  # Producer 0 and 1

# Demand per consumer
consumer_demand = np.array([10., 1., 2., 14., 4., 3., 1., 1.])

# Static weight vectors for each producer
producer_weights = np.array([
    [0.5, 0.1, 0., 0.3, 0., 0., 0.05, 0.05],  # Producer 0
    [0., 0., 0.3, 0.5, 0.1, 0.1, 0., 0.],     # Producer 1
])

# Preferences: each consumer's top 5 producers (use -1 for unconnected)
consumer_preferences = [
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [1, 0, -1, -1, -1],
]

# Track allocations
allocations = np.zeros((rounds, num_producers, num_consumers))

# Copy state
supply = producer_supply.copy()
demand = consumer_demand.copy()

for r in range(rounds):
    print(f"\n=== Round {r + 1} ===")

    # Each producer distributes supply according to static weights
    producer_offer = np.zeros((num_producers, num_consumers))
    for i in range(num_producers):
        if supply[i] > 0:
            producer_offer[i] = supply[i] * producer_weights[i]

    # Consumers take energy from preferred producers
    for j in range(num_consumers):
        if demand[j] <= 0:
            continue  # Already fulfilled

        for pref in consumer_preferences[j]:
            if pref == -1:
                continue

            offer = producer_offer[pref][j]
            if offer <= 0:
                continue

            take = min(offer, demand[j])

            # Allocate and update
            allocations[r][pref][j] += take
            producer_offer[pref][j] -= take
            supply[pref] -= take
            demand[j] -= take

            print(f"Consumer {j} takes {take:.1f} from Producer {pref}")

            if demand[j] <= 0:
                break  # Move to next consumer

# === Final Reporting ===
print("\n=== Final Allocations ===")
for r in range(rounds):
    print(f"\nRound {r + 1}:")
    for i in range(num_producers):
        for j in range(num_consumers):
            amount = allocations[r][i][j]
            if amount > 0:
                print(f"  Producer {i} → Consumer {j}: {amount:.1f}")

print("\nRemaining supply per producer:", supply)
print("Unmet demand per consumer:", demand)



=== Round 1 ===
Consumer 0 takes 7.5 from Producer 0
Consumer 1 takes 1.0 from Producer 0
Consumer 2 takes 2.0 from Producer 1
Consumer 3 takes 8.0 from Producer 1
Consumer 3 takes 4.5 from Producer 0
Consumer 4 takes 1.6 from Producer 1
Consumer 5 takes 1.6 from Producer 1
Consumer 6 takes 0.8 from Producer 0
Consumer 7 takes 0.8 from Producer 0

=== Round 2 ===
Consumer 0 takes 0.2 from Producer 0
Consumer 3 takes 1.4 from Producer 1
Consumer 3 takes 0.1 from Producer 0
Consumer 4 takes 0.3 from Producer 1
Consumer 5 takes 0.3 from Producer 1
Consumer 6 takes 0.0 from Producer 0
Consumer 7 takes 0.0 from Producer 0

=== Round 3 ===
Consumer 0 takes 0.1 from Producer 0
Consumer 4 takes 0.1 from Producer 1
Consumer 5 takes 0.1 from Producer 1
Consumer 6 takes 0.0 from Producer 0
Consumer 7 takes 0.0 from Producer 0

=== Round 4 ===
Consumer 0 takes 0.0 from Producer 0
Consumer 4 takes 0.1 from Producer 1
Consumer 5 takes 0.1 from Producer 1
Consumer 6 takes 0.0 from Producer 0
Consume

In [ ]:
num_consumers = 8
rounds = 5

# Supply per producer
producer_supply = np.array([15., 16.])  # Producer 0 and 1

# Demand per consumer
consumer_demand = np.array([10., 1., 2., 14., 4., 3., 1., 1.])

# Static weight vectors for each producer
producer_weights = np.array([
    [0.5, 0.1, 0., 0.3, 0., 0., 0.05, 0.05],  # Producer 0
    [0., 0., 0.3, 0.5, 0.1, 0.1, 0., 0.],     # Producer 1
])

# Preferences: each consumer's top 5 producers (use -1 for unconnected)
consumer_preferences = [
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [0, 1, -1, -1, -1], 
    [1, 0, -1, -1, -1], 
    [1, 0, -1, -1, -1],
]

# Track allocations
allocations = np.zeros((rounds, num_producers, num_consumers))

# Copy state
supply = producer_supply.copy()
demand = consumer_demand.copy()

for r in range(rounds):
    print(f"\n=== Round {r + 1} ===")

    # Each producer distributes supply according to static weights
    producer_offer = np.zeros((num_producers, num_consumers))
    for i in range(num_producers):
        if supply[i] > 0:
            producer_offer[i] = supply[i] * producer_weights[i]

    # Consumers take energy from preferred producers
    for j in range(num_consumers):
        if demand[j] <= 0:
            continue  # Already fulfilled

        for pref in consumer_preferences[j]:
            if pref == -1:
                continue

            offer = producer_offer[pref][j]
            if offer <= 0:
                continue

            take = min(offer, demand[j])

            # Allocate and update
            allocations[r][pref][j] += take
            producer_offer[pref][j] -= take
            supply[pref] -= take
            demand[j] -= take

            print(f"Consumer {j} takes {take:.1f} from Producer {pref}")

            if demand[j] <= 0:
                break  # Move to next consumer

# === Final Reporting ===
print("\n=== Final Allocations ===")
for r in range(rounds):
    print(f"\nRound {r + 1}:")
    for i in range(num_producers):
        for j in range(num_consumers):
            amount = allocations[r][i][j]
            if amount > 0:
                print(f"  Producer {i} → Consumer {j}: {amount:.1f}")

print("\nRemaining supply per producer:", supply)
print("Unmet demand per consumer:", demand)


In [2]:
import pandas as pd
import numpy as np

excess = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv')
deficit = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv')

producers = list(excess.columns[1:])
consumers = list(deficit.columns[1:])

In [ ]:
def generate_weights(producers, consumers):
    weights = {}
    for producer in producers:
        assigned_weights = np.random.dirichlet(np.ones(len(consumers)), size=1)[0]  # Ensures sum = 1
        weights[producer] = dict(zip(consumers, assigned_weights))
    return weights

weights = generate_weights(producers, consumers)

def generate_preferences(producers, consumers):
    preferences = {}
    for consumer in consumers:
        preferred_producers = np.random.choice(producers, size=5, replace=False)
        preference_values = np.random.permutation([5, 4, 3, 2, 1])
        preferences[consumer] = dict(zip(preferred_producers, preference_values))
    return preferences

preferences = generate_preferences(producers, consumers)


num_iterations = len(excess)
unredistributed_energy = []


In [9]:
timestamps = excess['timestamp'].values[12000:12000+94*30]
t = timestamps[60]

In [ ]:
excess.iloc[12060].drop('timestamp').to_dict()


{'zs_preislerova': 0.0097591374866565,
 'zs_komenskeho': 0.0,
 'ms_preislerova': 0.0098650279610302,
 'ms_pod_homolkou': 0.0023464378304447,
 'ms_vrchlickeho': 0.0018536020356289,
 'ms_drasarova': 0.0043899947469958,
 'ms_na_machovne': 0.0013770477141664,
 'zimni_stad': 0.0203570356331952,
 'parkovaci_dum': 0.0,
 'radnice': 0,
 'ms_tovarni': 0,
 'dum_pro_duchodce': 0,
 'plavecky_areal': 0,
 'pristavba_preislerova': 0.0114972}

In [14]:
deficit.iloc[12060].drop('timestamp').to_dict()

{'zs_preislerova': 0.0,
 'zs_komenskeho': 0.0004561419116203,
 'ms_preislerova': 0.0,
 'ms_pod_homolkou': 0.0,
 'ms_vrchlickeho': 0.0,
 'ms_drasarova': 0.0,
 'ms_na_machovne': 0.0,
 'zimni_stad': 0.0,
 'parkovaci_dum': 0.0027949328688059,
 'radnice': 0.0071286052099456,
 'ms_tovarni': 0.000428667194061,
 'dum_pro_duchodce': 6.41279529674551e-05,
 'plavecky_areal': 0.0410065207664489,
 'pristavba_preislerova': 0.0}

In [16]:
excess_t = excess.iloc[12060].drop('timestamp').to_dict()
deficit_t = deficit.iloc[1206].drop('timestamp').to_dict()

In [24]:
for consumer, demand in deficit_t.items():
    print(consumer, np.round(demand*1000, 2))

zs_preislerova 0.0
zs_komenskeho 5.36
ms_preislerova 0.0
ms_pod_homolkou 0.48
ms_vrchlickeho 0.0
ms_drasarova 0.0
ms_na_machovne 2.03
zimni_stad 14.03
parkovaci_dum 3.79
radnice 0.77
ms_tovarni 1.66
dum_pro_duchodce 0.02
plavecky_areal 31.96
pristavba_preislerova 0.0


In [29]:
t = 12060

In [70]:
excess_t = excess.iloc[t].drop('timestamp').to_dict()
deficit_t = deficit.iloc[t].drop('timestamp').to_dict()
round_unredistributed = 0

print(excess_t)
print(deficit_t)

{'zs_preislerova': 0.0097591374866565, 'zs_komenskeho': 0.0, 'ms_preislerova': 0.0098650279610302, 'ms_pod_homolkou': 0.0023464378304447, 'ms_vrchlickeho': 0.0018536020356289, 'ms_drasarova': 0.0043899947469958, 'ms_na_machovne': 0.0013770477141664, 'zimni_stad': 0.0203570356331952, 'parkovaci_dum': 0.0, 'radnice': 0, 'ms_tovarni': 0, 'dum_pro_duchodce': 0, 'plavecky_areal': 0, 'pristavba_preislerova': 0.0114972}
{'zs_preislerova': 0.0, 'zs_komenskeho': 0.0004561419116203, 'ms_preislerova': 0.0, 'ms_pod_homolkou': 0.0, 'ms_vrchlickeho': 0.0, 'ms_drasarova': 0.0, 'ms_na_machovne': 0.0, 'zimni_stad': 0.0, 'parkovaci_dum': 0.0027949328688059, 'radnice': 0.0071286052099456, 'ms_tovarni': 0.000428667194061, 'dum_pro_duchodce': 6.41279529674551e-05, 'plavecky_areal': 0.0410065207664489, 'pristavba_preislerova': 0.0}


In [71]:
deficit_t = {'radnice': 0.0071286052099456}

In [72]:

excess_t[sorted_producers[0][0]] * weights[sorted_producers[0][0]].get(consumer, 0)


np.float64(9.006071027087147e-05)

In [73]:

for round in range(5):
    
    for consumer, demand in deficit_t.items():
        if demand <= 0:
            #print(consumer, demand)
            continue

        sorted_producers = sorted(preferences.get(consumer, {}).items(), key=lambda x: -x[1])
        
        for producer, _ in sorted_producers:
            if excess_t[producer] <= 0:
                #print(producer, 'has null')
                continue
            
            allocated = excess_t[producer] * weights[producer].get(consumer, 0)
            consumed = min(allocated, demand)
            print(f'{producer}   gives:  {np.round(allocated* 10**5, 2)}, takes: {np.round(consumed * 10**5,2)}')
            
            excess_t[producer] -= consumed
            deficit_t[consumer] -= consumed
            
            if demand <= 0:
                break
        
    print(f'Demand after round {round} : {np.round(demand* 10**5, 2)}')



ms_vrchlickeho   gives:  9.01, takes: 9.01
ms_pod_homolkou   gives:  2.45, takes: 2.45
ms_preislerova   gives:  52.83, takes: 52.83
Demand after round 0 : 712.86
ms_vrchlickeho   gives:  8.57, takes: 8.57
ms_pod_homolkou   gives:  2.42, takes: 2.42
ms_preislerova   gives:  50.0, takes: 50.0
Demand after round 1 : 648.58
ms_vrchlickeho   gives:  8.15, takes: 8.15
ms_pod_homolkou   gives:  2.4, takes: 2.4
ms_preislerova   gives:  47.32, takes: 47.32
Demand after round 2 : 587.59
ms_vrchlickeho   gives:  7.76, takes: 7.76
ms_pod_homolkou   gives:  2.37, takes: 2.37
ms_preislerova   gives:  44.79, takes: 44.79
Demand after round 3 : 529.72
ms_vrchlickeho   gives:  7.38, takes: 7.38
ms_pod_homolkou   gives:  2.35, takes: 2.35
ms_preislerova   gives:  42.39, takes: 42.39
Demand after round 4 : 474.81


In [74]:
excess_t

{'zs_preislerova': 0.0097591374866565,
 'zs_komenskeho': 0.0,
 'ms_preislerova': np.float64(0.007491831272676784),
 'ms_pod_homolkou': np.float64(0.002226609546301364),
 'ms_vrchlickeho': np.float64(0.0014449812581231848),
 'ms_drasarova': 0.0043899947469958,
 'ms_na_machovne': 0.0013770477141664,
 'zimni_stad': 0.0203570356331952,
 'parkovaci_dum': 0.0,
 'radnice': 0,
 'ms_tovarni': 0,
 'dum_pro_duchodce': 0,
 'plavecky_areal': 0,
 'pristavba_preislerova': 0.0114972}

In [75]:
deficit_t

{'radnice': np.float64(0.004226959459943134)}

In [ ]:

for t in range(len(timestamps)):

    excess_t = excess.iloc[t].drop('timestamp').to_dict()
    deficit_t = deficit.iloc[t].drop('timestamp').to_dict()
    round_unredistributed = 0

    for r in range(5):
    
        for consumer, demand in deficit_t.items():
            if demand <= 0:
                continue

            sorted_producers = sorted(preferences.get(consumer, {}).items(), key=lambda x: -x[1])
            
            for producer, _ in sorted_producers:
                if excess_t[producer] <= 0:
                    continue
                
                allocated = excess_t[producer] * weights[producer].get(consumer, 0)
                consumed = min(allocated, demand)
                
                excess_t[producer] -= consumed
                deficit_t[consumer] -= consumed
                
                if demand <= 0:
                    break
    unredistributed_energy += np.sum(excess_t.values()) + np.sum(deficit_t.values())

In [76]:
def fitness(timestamps, excess, deficit, preferences, weights):
    res = 0

    for t in range(len(timestamps)):

        excess_t = excess.iloc[t].drop('timestamp').to_dict()
        deficit_t = deficit.iloc[t].drop('timestamp').to_dict()

        for r in range(5):
        
            for consumer, demand in deficit_t.items():
                if demand <= 0:
                    continue

                sorted_producers = sorted(preferences.get(consumer, {}).items(), key=lambda x: -x[1])
                
                for producer, _ in sorted_producers:
                    if excess_t[producer] <= 0:
                        continue
                    
                    allocated = excess_t[producer] * weights[producer].get(consumer, 0)
                    consumed = min(allocated, demand)
                    
                    excess_t[producer] -= consumed
                    deficit_t[consumer] -= consumed
                    
                    if demand <= 0:
                        break
                
        if demand > 0:
            round_unredistributed += demand
        
        res += np.sum(excess_t.values()) + np.sum(deficit_t.values())

    return res

In [77]:
EXCESS_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv'
DEFICIT_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv'

excess_df = pd.read_csv(EXCESS_CSV_PATH)
deficit_df = pd.read_csv(DEFICIT_CSV_PATH)

# Convert the timestamp column to datetime
excess_df['timestamp'] = pd.to_datetime(excess_df['timestamp'])
deficit_df['timestamp'] = pd.to_datetime(deficit_df['timestamp'])

# Extract the year and month from the timestamp
excess_df['year_month'] = excess_df['timestamp'].dt.to_period('M')
deficit_df['year_month'] = deficit_df['timestamp'].dt.to_period('M')

# Create 12 separate dataframes for each month (January to December)
excess_monthly = {}
deficit_monthly = {}

for i, month in enumerate(deficit_df['year_month'].unique()):
    
    # Filter the data for each month
    excess_monthly[i] = excess_df[excess_df['year_month'] == month].drop('year_month', axis=1)
    deficit_monthly[i] = deficit_df[deficit_df['year_month'] == month].drop('year_month', axis=1)


In [78]:
excess_monthly[11]

,timestamp,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
32060,2023-12-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
32061,2023-12-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
32062,2023-12-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
32063,2023-12-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
32064,2023-12-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35031,2023-12-31 22:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
35032,2023-12-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
35033,2023-12-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0
35034,2023-12-31 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0


In [ ]:
import numpy as np
import pandas as pd # Import pandas
import random
import copy
from tqdm import tqdm  # Optional: for progress bar
import os # To check if files exist

# --- Configuration ---
EXCESS_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv'
DEFICIT_CSV_PATH = '/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv'
NUM_PREFERENCES = 5  # Fixed number of producers each subscriber can connect to

# --- Genetic Algorithm Parameters ---
POPULATION_SIZE = 70
NUM_GENERATIONS = 500 # Adjust as needed
TOURNAMENT_SIZE = 5
MUTATION_RATE = 0.15 # Probability of mutating an individual (increased slightly)
CROSSOVER_RATE = 0.8 # Probability of performing crossover
ELITISM_COUNT = 3    # Keep the best N individuals directly

# --- Load Data and Set Simulation Parameters ---

print("Loading data...")
if not os.path.exists(EXCESS_CSV_PATH):
    raise FileNotFoundError(f"Excess data file not found at: {EXCESS_CSV_PATH}")
if not os.path.exists(DEFICIT_CSV_PATH):
    raise FileNotFoundError(f"Deficit data file not found at: {DEFICIT_CSV_PATH}")

try:
    excess = pd.read_csv(EXCESS_CSV_PATH)
    deficit = pd.read_csv(DEFICIT_CSV_PATH)

    # Convert the timestamp column to datetime
    excess['timestamp'] = pd.to_datetime(excess['timestamp'])
    deficit['timestamp'] = pd.to_datetime(deficit['timestamp'])

    # Extract the year and month from the timestamp
    excess['year_month'] = excess['timestamp'].dt.to_period('M')
    deficit['year_month'] = deficit['timestamp'].dt.to_period('M')

    # Create 12 separate dataframes for each month (January to December)
    excess_monthly = {}
    deficit_monthly = {}

    for i, month in enumerate(deficit['year_month'].unique()):
        
        # Filter the data for each month
        excess_monthly[i] = excess[excess['year_month'] == month].drop('year_month', axis=1)
        deficit_monthly[i] = deficit[deficit['year_month'] == month].drop('year_month', axis=1)
except Exception as e:
    print(f"Error loading CSV files: {e}")
    raise


Loading data...


In [80]:
month = 6

deficit_df = deficit_monthly[month]
excess_df = excess_monthly[month]


# --- Data Validation and Parameter Extraction ---
if excess_df.shape[0] != deficit_df.shape[0]:
    raise ValueError(f"Mismatch in number of timesteps (rows) between excess ({excess_df.shape[0]}) and deficit ({deficit_df.shape[0]}) data.")

# Assuming first column might be time or index, adjust if needed
# If first column IS producer/subscriber ID, handle accordingly
# Let's assume columns ARE the IDs after potential index col removal
if 'time' in excess_df.columns or 'timestamp' in excess_df.columns or 'Unnamed: 0' in excess_df.columns:
     # Drop potential time/index column if it exists - adjust column name as needed
     excess_df = excess_df.drop(columns=[col for col in ['time', 'timestamp', 'Unnamed: 0'] if col in excess_df.columns], errors='ignore')
if 'time' in deficit_df.columns or 'timestamp' in deficit_df.columns or 'Unnamed: 0' in deficit_df.columns:
     deficit_df = deficit_df.drop(columns=[col for col in ['time', 'timestamp', 'Unnamed: 0'] if col in deficit_df.columns], errors='ignore')



In [81]:
# Convert columns to numeric, coerce errors to NaN, then maybe fillna(0) or handle
excess_df = excess_df.apply(pd.to_numeric, errors='coerce').fillna(0)
deficit_df = deficit_df.apply(pd.to_numeric, errors='coerce').fillna(0)

NUM_PRODUCERS = excess_df.shape[1]
NUM_SUBSCRIBERS = deficit_df.shape[1]
SIMULATION_TIMESTEPS = excess_df.shape[0]

# Store producer/subscriber IDs if needed for interpretation (assumes columns are IDs)
PRODUCER_IDS = excess_df.columns.tolist()
SUBSCRIBER_IDS = deficit_df.columns.tolist()

# Convert DataFrames to NumPy arrays for efficiency in fitness function
PRODUCER_ENERGY_DATA = excess_df.to_numpy()
SUBSCRIBER_DEMAND_DATA = deficit_df.to_numpy()

print(f"Data loaded successfully:")
print(f"  Timesteps: {SIMULATION_TIMESTEPS}")
print(f"  Producers: {NUM_PRODUCERS}")
print(f"  Subscribers: {NUM_SUBSCRIBERS}")
print(f"  Preferences per subscriber: {NUM_PREFERENCES}")

if NUM_PRODUCERS < NUM_PREFERENCES:
    print(f"Warning: Number of producers ({NUM_PRODUCERS}) is less than the number of preferences ({NUM_PREFERENCES}). Subscribers will have duplicate or limited choices.")
    # Adjust NUM_PREFERENCES if strictly unique needed, or allow sampling with replacement later
    # NUM_PREFERENCES = NUM_PRODUCERS # Option: limit preferences


Data loaded successfully:
  Timesteps: 2976
  Producers: 14
  Subscribers: 14
  Preferences per subscriber: 5


In [ ]:
# --- Fitness Function ---
def fitness(weight_matrix, preference_matrix, producer_energy_data, subscriber_demand_data):
    """
    Calculates the total unredistributed energy over the simulation period.
    Lower is better. Uses ordered preferences (index 0 = highest priority).
    """

    total_unredistributed_energy = 0.0
    
    num_timesteps = producer_energy_data.shape[0]
    num_producers = weight_matrix.shape[0]
    num_subscribers = weight_matrix.shape[1]

    if preference_matrix.shape[0] != num_subscribers or preference_matrix.shape[1] != NUM_PREFERENCES:
         raise ValueError(f"Preference matrix shape mismatch. Expected ({num_subscribers}, {NUM_PREFERENCES}), Got {preference_matrix.shape}")
    if weight_matrix.shape[0] != num_producers or weight_matrix.shape[1] != num_subscribers:
         raise ValueError(f"Weight matrix shape mismatch. Expected ({num_producers}, {num_subscribers}), Got {weight_matrix.shape}")


    for t in range(num_timesteps):
        available_producer_energy = producer_energy_data[t, :].copy()
        remaining_subscriber_demand = subscriber_demand_data[t, :].copy()
        available_producer_energy[available_producer_energy < 1e-9] = 0
        remaining_subscriber_demand[remaining_subscriber_demand < 1e-9] = 0


        # --- Stage 1: Producer Allocation Intent (Based on Weights) ---

        intended_transfer = np.zeros((num_producers, num_subscribers))
        for p in range(num_producers):
            if available_producer_energy[p] > 1e-9:
                producer_weights = weight_matrix[p, :]
                total_weight = np.sum(producer_weights)
                if total_weight > 1e-9:
                    normalized_weights = producer_weights / total_weight
                    intended_transfer[p, :] = available_producer_energy[p] * normalized_weights

        # --- Stage 2: Subscriber Intake (Based on Ordered Preferences) ---
        temp_available_producer_energy = available_producer_energy.copy()

        for s in range(num_subscribers):
            if remaining_subscriber_demand[s] <= 1e-9:
                continue

            # Iterate through the subscriber's *ordered* preferred producers
            # preference_matrix[s, 0] is the highest priority, ..., preference_matrix[s, 4] is the lowest
            for pref_rank_idx in range(NUM_PREFERENCES):
                if remaining_subscriber_demand[s] <= 1e-9:
                    break # Demand met

                # Get the actual producer index from the preference list
                p_pref = preference_matrix[s, pref_rank_idx]

                # Ensure producer index is valid (e.g., if NUM_PREFERENCES > NUM_PRODUCERS was handled with placeholders)
                if p_pref < 0 or p_pref >= num_producers:
                    continue # Skip invalid producer index

                energy_intended_by_pref_p = intended_transfer[p_pref, s]
                energy_available_from_pref_p = temp_available_producer_energy[p_pref]

                energy_to_take = min(remaining_subscriber_demand[s],
                                     energy_intended_by_pref_p,
                                     energy_available_from_pref_p)

                if energy_to_take > 1e-9:
                    remaining_subscriber_demand[s] -= energy_to_take
                    temp_available_producer_energy[p_pref] -= energy_to_take
                    # Ensure non-negative after subtraction
                    remaining_subscriber_demand[s] = max(0, remaining_subscriber_demand[s])
                    temp_available_producer_energy[p_pref] = max(0, temp_available_producer_energy[p_pref])


        # --- Step 3: Calculate Unmet Demand for this Timestep ---
        unredistributed_this_step = np.sum(remaining_subscriber_demand)
        total_unredistributed_energy += unredistributed_this_step

    # Add a penalty for invalid matrices if necessary (e.g., weights not summing to 1)
    # Although initialization and mutation try to maintain validity.
    # Could add checks here for debugging.

    return total_unredistributed_energy

# --- Genetic Algorithm Components ---

class Individual:
    """Represents a potential solution (weight and preference matrices)."""
    def __init__(self, weight_matrix=None, preference_matrix=None):
        self.weight_matrix = weight_matrix
        self.preference_matrix = preference_matrix # Stores ORDERED list of preferred producer indices
        self.fitness = float('inf')

        if self.weight_matrix is None:
            self.initialize_weights()
        if self.preference_matrix is None:
            self.initialize_preferences()

    def initialize_weights(self):
        """Initializes weights randomly, ensuring each producer's weights sum to 1."""
        weights = np.random.rand(NUM_PRODUCERS, NUM_SUBSCRIBERS)
        row_sums = weights.sum(axis=1, keepdims=True)
        row_sums[row_sums < 1e-9] = 1.0
        self.weight_matrix = weights / row_sums

    def initialize_preferences(self):
        """
        Initializes preferences randomly. Each subscriber gets an ORDERED list
        of NUM_PREFERENCES unique producer indices. Order implies priority.
        """
        prefs = np.zeros((NUM_SUBSCRIBERS, NUM_PREFERENCES), dtype=int)
        producer_indices = list(range(NUM_PRODUCERS))

        k = min(NUM_PREFERENCES, NUM_PRODUCERS) # How many unique producers can we actually pick?

        for s in range(NUM_SUBSCRIBERS):
            if NUM_PRODUCERS == 0: # Handle edge case
                 prefs[s,:] = -1 # Or some other indicator of no valid producers
                 continue

            # Sample k unique producers
            chosen_producers = random.sample(producer_indices, k)
            # Shuffle them to get a random initial order (priority)
            random.shuffle(chosen_producers)
            prefs[s, :k] = chosen_producers

            # Handle if NUM_PREFERENCES > NUM_PRODUCERS
            if k < NUM_PREFERENCES:
                # Fill remaining slots. Options:
                # 1. Use placeholder (-1)
                # 2. Sample *with replacement* from existing producers
                # Option 2 implemented here:
                remaining_prefs = np.random.choice(producer_indices, NUM_PREFERENCES - k, replace=True)
                prefs[s, k:] = remaining_prefs
                # Could also shuffle the entire list again after filling
                # random.shuffle(prefs[s, :])

        self.preference_matrix = prefs

    def calculate_fitness(self):
        """Calculates and stores the fitness of this individual."""
        try:
            self.fitness = fitness(self.weight_matrix, self.preference_matrix,
                                   PRODUCER_ENERGY_DATA, SUBSCRIBER_DEMAND_DATA)
        except ValueError as e:
            print(f"Fitness calculation error: {e}")
            print("Weight Matrix sample (first 5x5):")
            print(self.weight_matrix[:5,:5])
            print("Preference Matrix sample (first 5 rows):")
            print(self.preference_matrix[:5,:])
            self.fitness = float('inf') # Assign worst fitness on error


def initialize_population(size):
    """Creates the initial population of individuals."""
    return [Individual() for _ in range(size)]

def tournament_selection(population, k=TOURNAMENT_SIZE):
    """Selects an individual using tournament selection."""
    # Ensure population is not empty and k is not larger than population size
    if not population:
        raise ValueError("Cannot perform selection on an empty population.")
    k = min(k, len(population))
    if k <= 0:
        raise ValueError("Tournament size k must be positive.")

    tournament = random.sample(population, k)
    winner = min(tournament, key=lambda ind: ind.fitness)
    return winner

def crossover(parent1, parent2):
    """Performs crossover between two parents to create two children."""
    # Use deepcopy to avoid modifying parents inadvertently
    child1 = Individual(copy.deepcopy(parent1.weight_matrix), copy.deepcopy(parent1.preference_matrix))
    child2 = Individual(copy.deepcopy(parent2.weight_matrix), copy.deepcopy(parent2.preference_matrix))

    if random.random() < CROSSOVER_RATE:
        # --- Crossover for Weight Matrix (Example: Single point row crossover) ---
        if NUM_PRODUCERS > 1:
            crossover_point_p = random.randint(1, NUM_PRODUCERS - 1)
            child1.weight_matrix[crossover_point_p:] = parent2.weight_matrix[crossover_point_p:]
            child2.weight_matrix[crossover_point_p:] = parent1.weight_matrix[crossover_point_p:]
            # Optional: Re-normalize rows after crossover (might be good practice)
            for r in range(NUM_PRODUCERS):
                 for child_w in [child1.weight_matrix, child2.weight_matrix]:
                    row_sum = np.sum(child_w[r, :])
                    if row_sum > 1e-9:
                         child_w[r, :] /= row_sum
                    elif NUM_SUBSCRIBERS > 0: # Avoid division by zero if re-initializing
                         child_w[r, :] = 1.0 / NUM_SUBSCRIBERS # Re-initialize row uniformly


        # --- Crossover for Preference Matrix (Example: Single point subscriber crossover) ---
        if NUM_SUBSCRIBERS > 1:
            crossover_point_s = random.randint(1, NUM_SUBSCRIBERS - 1)
            child1.preference_matrix[crossover_point_s:] = parent2.preference_matrix[crossover_point_s:]
            child2.preference_matrix[crossover_point_s:] = parent1.preference_matrix[crossover_point_s:]

    return child1, child2


def mutate(individual):
    """Performs mutation on an individual."""

    # --- Mutate Weight Matrix ---
    if random.random() < MUTATION_RATE and NUM_PRODUCERS > 0 and NUM_SUBSCRIBERS > 0:
        p = random.randint(0, NUM_PRODUCERS - 1)
        s = random.randint(0, NUM_SUBSCRIBERS - 1)
        individual.weight_matrix[p, s] += random.gauss(0, 0.1) # Adjust noise level
        individual.weight_matrix[p, s] = max(0, individual.weight_matrix[p, s]) # Ensure non-negative

        # Re-normalize the mutated row
        row_sum = np.sum(individual.weight_matrix[p, :])
        if row_sum > 1e-9:
            individual.weight_matrix[p, :] /= row_sum
        elif NUM_SUBSCRIBERS > 0:
             individual.weight_matrix[p, :] = 1.0 / NUM_SUBSCRIBERS # Re-initialize uniformly


    # --- Mutate Preference Matrix ---
    if random.random() < MUTATION_RATE and NUM_SUBSCRIBERS > 0 and NUM_PREFERENCES > 0:
         s_to_mutate = random.randint(0, NUM_SUBSCRIBERS - 1)

         # Mutation type 1: Swap order of two preferences (changes priority)
         if random.random() < 0.5 and NUM_PREFERENCES > 1: # Need at least 2 prefs to swap
             idx1, idx2 = random.sample(range(NUM_PREFERENCES), 2)
             # Swap elements at these indices
             individual.preference_matrix[s_to_mutate, [idx1, idx2]] = \
                 individual.preference_matrix[s_to_mutate, [idx2, idx1]]
         # Mutation type 2: Replace one preference with a new producer
         elif NUM_PRODUCERS > NUM_PREFERENCES: # Only possible if there are producers *not* currently preferred
             pref_idx_to_replace = random.randint(0, NUM_PREFERENCES - 1)
             current_prefs = set(individual.preference_matrix[s_to_mutate, :]) # Use set for efficient lookup
             possible_new_producers = list(range(NUM_PRODUCERS))
             available_new_producers = [p for p in possible_new_producers if p not in current_prefs]

             if available_new_producers:
                 new_producer = random.choice(available_new_producers)
                 individual.preference_matrix[s_to_mutate, pref_idx_to_replace] = new_producer
         # Add more mutation strategies if needed (e.g. shuffle entire preference list)


# --- Main GA Loop ---
def run_genetic_algorithm():
    """Executes the genetic algorithm."""
    if NUM_PRODUCERS == 0 or NUM_SUBSCRIBERS == 0:
         print("Error: No producers or subscribers found in the data. Cannot run GA.")
         return None, None

    population = initialize_population(POPULATION_SIZE)

    print("Calculating initial fitness...")
    # Use tqdm for progress bar if available
    for ind in tqdm(population, desc="Initial Fitness Calc"):
        ind.calculate_fitness()

    best_overall_fitness = float('inf')
    best_overall_individual = None

    print(f"\nStarting GA ({NUM_GENERATIONS} generations, {POPULATION_SIZE} population)")
    generation_iterator = tqdm(range(NUM_GENERATIONS), desc="Generations") if 'tqdm' in globals() else range(NUM_GENERATIONS)

    for generation in generation_iterator:
        # Sort population by fitness (ascending order - lower is better)
        population.sort(key=lambda ind: ind.fitness)

        # Track best fitness
        current_best_fitness = population[0].fitness
        if current_best_fitness < best_overall_fitness:
            best_overall_fitness = current_best_fitness
            best_overall_individual = copy.deepcopy(population[0])
            update_message = f"Gen {generation+1}/{NUM_GENERATIONS} - New Best Fitness: {best_overall_fitness:.4f}"
            if 'tqdm' in globals(): generation_iterator.set_postfix_str(update_message)
            print(update_message) # Print regardless of tqdm
        elif (generation + 1) % 10 == 0: # Periodic update even if no improvement
            update_message = f"Gen {generation+1}/{NUM_GENERATIONS} - Crnt Best: {population[0].fitness:.4f} (Ovrll Best: {best_overall_fitness:.4f})"
            if 'tqdm' in globals(): generation_iterator.set_postfix_str(update_message)
            else: print(update_message)


        # Create next generation
        new_population = []

        # Elitism
        if ELITISM_COUNT > 0:
            # Ensure elitism count doesn't exceed population size
            actual_elitism_count = min(ELITISM_COUNT, len(population))
            new_population.extend(population[:actual_elitism_count])

        # Fill the rest
        while len(new_population) < POPULATION_SIZE:
            if not population: break # Safety break if population somehow becomes empty
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)

            child1, child2 = crossover(parent1, parent2)

            mutate(child1)
            mutate(child2)

            # Calculate fitness for new children *before* adding to selection pool for next gen
            child1.calculate_fitness()
            new_population.append(child1)

            if len(new_population) < POPULATION_SIZE:
                 child2.calculate_fitness()
                 new_population.append(child2)

        population = new_population
        if not population: # Safety check
            print("Error: Population became empty during generation loop.")
            break


    # Final check and return
    if not population and not best_overall_individual:
         print("Error: GA finished without any valid individuals.")
         return None, None

    # If population exists, one last sort might be needed if elitism wasn't perfect
    if population:
        population.sort(key=lambda ind: ind.fitness)
        if best_overall_individual is None or population[0].fitness < best_overall_fitness :
             best_overall_individual = population[0]
             best_overall_fitness = population[0].fitness


    print("\nGA Finished.")
    if best_overall_individual:
        print(f"Best overall fitness found: {best_overall_fitness:.4f}")
        return best_overall_individual.weight_matrix, best_overall_individual.preference_matrix
    else:
        print("Warning: No best individual found (possibly due to errors or empty population).")
        return None, None

# --- Run the Algorithm ---


Data loaded successfully:
  Timesteps: 2976
  Producers: 14
  Subscribers: 14
  Preferences per subscriber: 5


In [ ]:
optimal_weight_matrix, optimal_preference_matrix = run_genetic_algorithm()

if optimal_weight_matrix is not None and optimal_preference_matrix is not None:
    print("\nOptimal Weight Matrix (Top 5x5):")
    print(np.round(optimal_weight_matrix[:5,:5], 3))
    print("\nOptimal Preference Matrix (Top 5 rows):")
    # Display corresponding producer IDs if available and useful
    # pref_df = pd.DataFrame(optimal_preference_matrix, index=SUBSCRIBER_IDS[:optimal_preference_matrix.shape[0]])
    # You might want to map indices back to actual producer IDs here if PRODUCER_IDS was stored
    # Example: map_ids_func = np.vectorize(lambda x: PRODUCER_IDS[x] if 0 <= x < len(PRODUCER_IDS) else 'Invalid')
    # mapped_prefs = map_ids_func(optimal_preference_matrix)
    # print(pd.DataFrame(mapped_prefs[:5,:], index=SUBSCRIBER_IDS[:5]))
    print(optimal_preference_matrix[:5,:]) # Print indices for now

    # --- Optional: Save results ---
    # np.save("optimal_weight_matrix.npy", optimal_weight_matrix)
    # np.save("optimal_preference_matrix.npy", optimal_preference_matrix)
    # print("\nOptimal matrices saved to .npy files.")

    # You can also save to CSV
    weight_df = pd.DataFrame(optimal_weight_matrix, columns=SUBSCRIBER_IDS, index=PRODUCER_IDS)
    pref_df = pd.DataFrame(optimal_preference_matrix, index=SUBSCRIBER_IDS) # Columns are Rank 1..5
    weight_df.to_csv("optimal_weight_matrix.csv")
    pref_df.to_csv("optimal_preference_matrix.csv")
    # print("Optimal matrices saved to .csv files.")

Calculating initial fitness...


Initial Fitness Calc: 100%|██████████| 70/70 [00:18<00:00,  3.89it/s]



Starting GA (500 generations, 70 population)


Generations:   0%|          | 0/500 [00:00<?, ?it/s, Gen 1/500 - New Best Fitness: 147.5797]

Gen 1/500 - New Best Fitness: 147.5797


...

Gen 496/500 - New Best Fitness: 100.6596


Generations: 100%|█████████▉| 499/500 [2:23:46<00:17, 17.29s/it, Gen 500/500 - New Best Fitness: 100.6396]

Gen 500/500 - New Best Fitness: 100.6396


Generations: 100%|██████████| 500/500 [2:24:03<00:00, 17.29s/it, Gen 500/500 - New Best Fitness: 100.6396]


GA Finished.
Best overall fitness found: 100.6396

Optimal Weight Matrix (Top 5x5):
[[0.    0.042 0.    0.    0.   ]
 [0.11  0.072 0.    0.088 0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]]

Optimal Preference Matrix (Top 5 rows):
[[ 2  6  8  5  7]
 [ 5  0  6  4 13]
 [ 3  9  6  7  2]
 [ 6  7  4  2  5]
 [ 6 11  0  2  8]]


In [91]:
print("\nOptimal Weight Matrix (Top 5x5):")
print(np.round(optimal_weight_matrix, 3))
print("\nOptimal Preference Matrix (Top 5 rows):")
# Display corresponding producer IDs if available and useful
# pref_df = pd.DataFrame(optimal_preference_matrix, index=SUBSCRIBER_IDS[:optimal_preference_matrix.shape[0]])
# You might want to map indices back to actual producer IDs here if PRODUCER_IDS was stored
# Example: map_ids_func = np.vectorize(lambda x: PRODUCER_IDS[x] if 0 <= x < len(PRODUCER_IDS) else 'Invalid')
# mapped_prefs = map_ids_func(optimal_preference_matrix)
# print(pd.DataFrame(mapped_prefs[:5,:], index=SUBSCRIBER_IDS[:5]))
print(optimal_preference_matrix)


Optimal Weight Matrix (Top 5x5):
[[0.    0.042 0.    0.    0.    0.    0.    0.    0.    0.958 0.    0.
  0.    0.   ]
 [0.11  0.072 0.    0.088 0.    0.097 0.034 0.    0.283 0.137 0.079 0.
  0.042 0.058]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  1.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.01  0.391 0.222 0.006 0.103
  0.267 0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.145 0.    0.    0.    0.    0.    0.78  0.    0.048 0.    0.027
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.068 0.    0.    0.154 0.015
  0.762 0.001]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.084 0.    0.001 0.
  0.915 0.   ]
 [0.    0.    0.    0.084 0.    0.    0.    0.559 0.    0.075 0.    0.
  0.114 0.168]
 [0.012 0.087 0.076 0.174 0.017 0.064 0.013 0.203 0.037 0.162 0.032 0.072
  0.    0.051]
 [0.113 0.039 0.065 0.086 0.025 0.02  0.033 0.006 0.062 0.111 0.135 0.116
  0.038 0.151]
 [0.0

In [93]:
excess_df

,timestamp,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
17372,2023-07-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
17373,2023-07-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
17374,2023-07-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
17375,2023-07-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
17376,2023-07-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20343,2023-07-31 22:45:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
20344,2023-07-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0.0
20345,2023-07-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.000007,0.0,0.0,0.0,0,0,0,0,0.0
20346,2023-07-31 23:30:00,0.0,0.0,0.0,0.0,0.0,0.000014,0.0,0.0,0.0,0,0,0,0,0.0


In [92]:
fitness(optimal_weight_matrix, optimal_preference_matrix, excess_df, deficit_df)

InvalidIndexError: (0, slice(None, None, None))

In [84]:
deficit_df

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
17372,0.000785,0.002508,0.000443,0.001529,0.000061,0.000159,0.0,0.000717,0.002726,0.000356,0.000651,0.000058,0.022678,0.0
17373,0.000981,0.002502,0.000267,0.000911,0.000136,0.000153,0.0,0.000730,0.002411,0.000416,0.000560,0.000045,0.023199,0.0
17374,0.000785,0.001828,0.000596,0.000359,0.000090,0.000171,0.0,0.000704,0.002586,0.000475,0.000609,0.000070,0.022930,0.0
17375,0.000867,0.001620,0.000333,0.000702,0.000085,0.000105,0.0,0.000776,0.002557,0.001307,0.000579,0.000064,0.020433,0.0
17376,0.000785,0.002166,0.000364,0.000664,0.000021,0.000006,0.0,0.001080,0.002762,0.000713,0.000231,0.000042,0.022689,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20343,0.000589,0.001595,0.000366,0.000115,0.000056,0.000106,0.0,0.002367,0.005488,0.000000,0.000216,0.000114,0.027565,0.0
20344,0.000670,0.002167,0.000243,0.000267,0.000022,0.000008,0.0,0.002286,0.006280,0.000000,0.000206,0.000088,0.024151,0.0
20345,0.000393,0.001719,0.000261,0.000413,0.000040,0.000000,0.0,0.002352,0.004810,0.000000,0.000181,0.000095,0.028021,0.0
20346,0.000589,0.001870,0.000336,0.000282,0.000054,0.000000,0.0,0.002341,0.005994,0.000535,0.000194,0.000084,0.024175,0.0


In [85]:
deficit_df = deficit_monthly[month]
excess_df = excess_monthly[month]



In [86]:
deficit_df[[ col for col in excess_df.columns[1:] ]]

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
17372,0.000785,0.002508,0.000443,0.001529,0.000061,0.000159,0.0,0.000717,0.002726,0.000356,0.000651,0.000058,0.022678,0.0
17373,0.000981,0.002502,0.000267,0.000911,0.000136,0.000153,0.0,0.000730,0.002411,0.000416,0.000560,0.000045,0.023199,0.0
17374,0.000785,0.001828,0.000596,0.000359,0.000090,0.000171,0.0,0.000704,0.002586,0.000475,0.000609,0.000070,0.022930,0.0
17375,0.000867,0.001620,0.000333,0.000702,0.000085,0.000105,0.0,0.000776,0.002557,0.001307,0.000579,0.000064,0.020433,0.0
17376,0.000785,0.002166,0.000364,0.000664,0.000021,0.000006,0.0,0.001080,0.002762,0.000713,0.000231,0.000042,0.022689,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20343,0.000589,0.001595,0.000366,0.000115,0.000056,0.000106,0.0,0.002367,0.005488,0.000000,0.000216,0.000114,0.027565,0.0
20344,0.000670,0.002167,0.000243,0.000267,0.000022,0.000008,0.0,0.002286,0.006280,0.000000,0.000206,0.000088,0.024151,0.0
20345,0.000393,0.001719,0.000261,0.000413,0.000040,0.000000,0.0,0.002352,0.004810,0.000000,0.000181,0.000095,0.028021,0.0
20346,0.000589,0.001870,0.000336,0.000282,0.000054,0.000000,0.0,0.002341,0.005994,0.000535,0.000194,0.000084,0.024175,0.0


In [87]:
optimal = 0
for time_stamp in deficit_df['timestamp']:
    df1 =deficit_df[deficit_df['timestamp'] == time_stamp][[col for col in deficit_df.columns[1:] ]]
    df2 = excess_df[excess_df['timestamp'] == time_stamp][[col for col in excess_df.columns[1:] ]]

    optimal += np.max([ 
                        df1.sum().sum() - df2[[col for col in excess_df.columns[1:]]].sum().sum(),
                    0])


In [88]:
print(optimal)

97.37187323577675


In [1]:
import pandas as pd
import numpy as np
import pulp
import pickle

# -----------------------------
# Data Inputs
# -----------------------------
# Load energy data (15-min interval time-series)
excess = excess_df.copy()
deficit = deficit_df.copy()

# Convert DataFrames to numpy arrays
producer_supply = excess.values    # shape: (2880, num_producers)
consumer_demand = deficit.values     # shape: (2880, num_consumers)

num_timesteps, num_producers = producer_supply.shape
_, num_consumers = consumer_demand.shape

# -----------------------------
# MILP Model Setup
# -----------------------------
model = pulp.LpProblem("Energy_Distribution", pulp.LpMinimize)

# Decision Variables:
# x[p,s,t] is the amount of energy allocated from producer p to consumer s at time t.
# It is continuous and nonnegative.
x = pulp.LpVariable.dicts("x", 
        ((p, s, t) for p in range(num_producers) 
                     for s in range(num_consumers) 
                     for t in range(num_timesteps)),
        lowBound=0, cat='Continuous')

# y[p,s] is a binary variable indicating whether consumer s is connected to producer p.
y = pulp.LpVariable.dicts("y", 
        ((p, s) for p in range(num_producers) 
                 for s in range(num_consumers)),
        cat='Binary')

# For this example, we initialize random preferences for each potential connection.
# Preferences: 5 = highest priority, 1 = lowest. (y[p,s]==0 will imply no connection.)
np.random.seed(0)
preference = {(p, s): np.random.randint(1, 6) 
              for p in range(num_producers) for s in range(num_consumers)}

# For later use in the objective, we define a penalty for lower priority use.
# Here, we set penalty = (6 - preference). So a connection with preference 5 incurs a penalty of 1.
penalty = {(p, s): 6 - preference[(p, s)] 
           for p in range(num_producers) for s in range(num_consumers)}

# A big M value for linking allocations to connections.
M = max(consumer_demand.max(), producer_supply.max())

# -----------------------------
# Model Constraints
# -----------------------------
# 1. Producer supply: At every time step, the sum of energy allocated from a producer cannot exceed its available supply.
for p in range(num_producers):
    for t in range(num_timesteps):
        model += (pulp.lpSum(x[(p, s, t)] for s in range(num_consumers))
                  <= producer_supply[t, p]), f"ProducerSupply_p{p}_t{t}"

# 2. Consumer demand: At every time step, the sum of energy received by a consumer cannot exceed its demand.
for s in range(num_consumers):
    for t in range(num_timesteps):
        model += (pulp.lpSum(x[(p, s, t)] for p in range(num_producers))
                  <= consumer_demand[t, s]), f"ConsumerDemand_s{s}_t{t}"

# 3. Connection enforcement: Energy can be allocated from producer p to consumer s only if there is a connection.
for p in range(num_producers):
    for s in range(num_consumers):
        for t in range(num_timesteps):
            model += x[(p, s, t)] <= M * y[(p, s)], f"Connection_p{p}_s{s}_t{t}"

# 4. Limit on connections: Each consumer may only be connected to up to 5 producers.
for s in range(num_consumers):
    model += (pulp.lpSum(y[(p, s)] for p in range(num_producers)) <= 5), f"MaxConnections_s{s}"

# 5. (Optional) To favor high-preference connections, we include a penalty term in the objective.
#    (An alternative would be to enforce strict ordering constraints, but that adds complexity.)

# -----------------------------
# Objective Function
# -----------------------------
# Our goal is to minimize the total unredistributed energy over the month.
# This is the sum of:
#   a) Producer leftover energy: available supply minus allocated energy.
#   b) Consumer unmet demand: demand minus energy received.
# In addition, we add a penalty weighted by the preference (i.e. using a low-priority connection incurs extra cost).
leftover_supply = pulp.lpSum(
    producer_supply[t, p] - pulp.lpSum(x[(p, s, t)] for s in range(num_consumers))
    for p in range(num_producers) for t in range(num_timesteps))

unmet_demand = pulp.lpSum(
    consumer_demand[t, s] - pulp.lpSum(x[(p, s, t)] for p in range(num_producers))
    for s in range(num_consumers) for t in range(num_timesteps))

penalty_term = pulp.lpSum(
    penalty[(p, s)] * x[(p, s, t)]
    for p in range(num_producers) for s in range(num_consumers) for t in range(num_timesteps))

model += leftover_supply + unmet_demand + penalty_term, "Total_Unredistributed_Energy"

# -----------------------------
# Solve the MILP
# -----------------------------
model.solve(pulp.PULP_CBC_CMD())

# -----------------------------
# Extract and Process Results
# -----------------------------
# Allocation matrix (x): dimensions (time, producer, consumer)
optimal_x = np.zeros((num_timesteps, num_producers, num_consumers))
for p in range(num_producers):
    for s in range(num_consumers):
        for t in range(num_timesteps):
            optimal_x[t, p, s] = pulp.value(x[(p, s, t)])

# Connection matrix (y): dimensions (producer, consumer)
optimal_y = np.zeros((num_producers, num_consumers))
for p in range(num_producers):
    for s in range(num_consumers):
        optimal_y[p, s] = pulp.value(y[(p, s)])

# Compute total unredistributed energy
total_unredistributed_energy = 0
for t in range(num_timesteps):
    for p in range(num_producers):
        leftover = producer_supply[t, p] - sum(optimal_x[t, p, s] for s in range(num_consumers))
        total_unredistributed_energy += leftover
    for s in range(num_consumers):
        unmet = consumer_demand[t, s] - sum(optimal_x[t, p, s] for p in range(num_producers))
        total_unredistributed_energy += unmet

# Breakdown: Aggregate unmet demand per consumer and leftover energy per producer
consumer_unmet = {s: 0 for s in range(num_consumers)}
producer_leftover = {p: 0 for p in range(num_producers)}

for s in range(num_consumers):
    for t in range(num_timesteps):
        consumer_unmet[s] += consumer_demand[t, s] - sum(optimal_x[t, p, s] for p in range(num_producers))
for p in range(num_producers):
    for t in range(num_timesteps):
        producer_leftover[p] += producer_supply[t, p] - sum(optimal_x[t, p, s] for s in range(num_consumers))

# -----------------------------
# Output the Results to a File
# -----------------------------
output = {
    "optimal_weight_matrix": optimal_x,           # x serves as the allocation (proxy for weight distribution)
    "optimal_connection_matrix": optimal_y,         # y indicates which connections (i.e. preferences) are used
    "total_unredistributed_energy": total_unredistributed_energy,
    "consumer_unmet_demand": consumer_unmet,
    "producer_leftover_energy": producer_leftover
}

with open("energy_distribution_output.pkl", "wb") as f:
    pickle.dump(output, f)

print("MILP model solved. Output saved to energy_distribution_output.pkl")


NameError: name 'excess_df' is not defined